**This notebook will provie split logic for datasets. We'll use training, validation and test sets here. An assumption is made that we will use stochastic gradient descent thus observations will be randomized and saved accordingly.**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pyprojroot import here
from sklearn.preprocessing import StandardScaler


In [2]:
#importing encoded unscaled data

unscaled_processed_df = pd.read_csv(here('data/processed/encoded_unscaled_df.csv'))
print(unscaled_processed_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Air temperature      10000 non-null  float64
 1   Process temperature  10000 non-null  float64
 2   Rotational speed     10000 non-null  int64  
 3   Torque               10000 non-null  float64
 4   Tool wear            10000 non-null  int64  
 5   quality_variation    10000 non-null  int64  
 6   Machine failure      10000 non-null  int64  
 7   TWF                  10000 non-null  int64  
 8   HDF                  10000 non-null  int64  
 9   PWF                  10000 non-null  int64  
 10  OSF                  10000 non-null  int64  
 11  RNF                  10000 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 937.6 KB
None


In [3]:
np.random.seed(7)
indices = unscaled_processed_df.index.array
print(indices)
#Shuffling because we'll use SGD for these training sets
shuffled_indices = np.random.permutation(indices)
unscaled_processed_df = unscaled_processed_df.reindex(labels=shuffled_indices,axis=0)
print(unscaled_processed_df.head(3))

<NumpyExtensionArray>
[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
 ...
 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999]
Length: 10000, dtype: int64
      Air temperature  Process temperature  Rotational speed  Torque  \
1977            298.1                307.7              1633    30.7   
3880            302.5                311.6              1595    29.5   
52              298.8                309.0              1497    43.8   

      Tool wear  quality_variation  Machine failure  TWF  HDF  PWF  OSF  RNF  
1977        145                  1                0    0    0    0    0    0  
3880         35                  0                0    0    0    0    0    0  
52          147                  2                0    0    0    0    0    0  


In [4]:
X = unscaled_processed_df.loc[:,['quality_variation','Air temperature','Process temperature','Rotational speed','Torque','Tool wear']]
Y = unscaled_processed_df.loc[:,['Machine failure','TWF','HDF','PWF','OSF','RNF']]
print(X.shape)
print(Y.shape)


(10000, 6)
(10000, 6)


### Cheatsheet: two-step split params
_First split_: `test_size1 = test`  
_Second split (on remainder)_: `test_size2 = val / (1 - test)`

| Final split (train/val/test) | `test_size1` (for test) | `test_size2` (for val) |
|---|---:|---:|
| 60/20/20 | 0.2000 | 0.2500 |
| 70/15/15 | 0.1500 | 0.1765 |
| 75/15/10 | 0.1000 | 0.1667 |
| 80/10/10 | 0.1000 | 0.1111 |
| 80/15/5  | 0.0500 | 0.1579 |
| 65/20/15 | 0.1500 | 0.2353 |
| 50/25/25 | 0.2500 | 0.3333 |


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1000,random_state=1234)

X_train, X_val, Y_train, Y_val =  train_test_split(X_train,Y_train, test_size=0.1667, random_state=1234)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(f'y values are,{Y_train.shape},{Y_val.shape},{Y_test.shape}')

(7499, 6)
(1501, 6)
(1000, 6)
y values are,(7499, 6),(1501, 6),(1000, 6)


In [6]:
#Saving Training, Validation, Test splits
X_train.to_csv(here('data/processed/X_train.csv'), index=False)
Y_train.to_csv(here('data/processed/Y_train.csv'), index=False)

X_val.to_csv(here('data/processed/X_val.csv'), index=False)
Y_val.to_csv(here('data/processed/Y_val.csv'), index=False)

X_test.to_csv(here('data/processed/X_test.csv'), index=False)
Y_test.to_csv(here('data/processed/Y_test.csv'), index=False)


In [7]:
# Create std scaler for X
scaler = StandardScaler().fit(X_train)

In [8]:
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)

print(X_train_scaled.mean())
print('-----------------')
print(X_train_scaled.std())


quality_variation      5.495597e-17
Air temperature        1.912799e-14
Process temperature   -6.398817e-15
Rotational speed       1.478126e-16
Torque                -2.529869e-16
Tool wear             -7.864388e-17
dtype: float64
-----------------
quality_variation      1.000067
Air temperature        1.000067
Process temperature    1.000067
Rotational speed       1.000067
Torque                 1.000067
Tool wear              1.000067
dtype: float64


In [9]:
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)

print(X_val_scaled.mean())
print('-----------------')
print(X_val_scaled.std())

quality_variation      0.030265
Air temperature        0.004923
Process temperature   -0.013613
Rotational speed       0.014343
Torque                -0.029444
Tool wear             -0.018978
dtype: float64
-----------------
quality_variation      1.029998
Air temperature        0.984027
Process temperature    0.985851
Rotational speed       0.948562
Torque                 0.985027
Tool wear              0.980927
dtype: float64


In [10]:
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print(X_test_scaled.mean())
print('-----------------')
print(X_test_scaled.std())

quality_variation     -0.012053
Air temperature        0.002965
Process temperature    0.021866
Rotational speed       0.026043
Torque                -0.008779
Tool wear              0.016080
dtype: float64
-----------------
quality_variation      0.984752
Air temperature        0.999694
Process temperature    1.018121
Rotational speed       1.000820
Torque                 1.020916
Tool wear              1.023871
dtype: float64


In [11]:
#Saving Scaled Training, Validation, Test splits
X_train_scaled.to_csv(here('data/processed/X_train_scaled.csv'), index=False)
Y_train.to_csv(here('data/processed/Y_train.csv'), index=False)

X_val_scaled.to_csv(here('data/processed/X_val_scaled.csv'), index=False)
Y_val.to_csv(here('data/processed/Y_val.csv'), index=False)

X_test_scaled.to_csv(here('data/processed/X_test_scaled.csv'), index=False)
Y_test.to_csv(here('data/processed/Y_test.csv'), index=False)

Size Before Preprocessing: One dataset, (10000, 12)
| Split         | X Shape   | Y Shape   |
|---------------|-----------|-----------|
| Training      | (7499, 6) | (7499, 6) |
| Validation    | (1501, 6) | (1501, 6) |
| Test          | (1000, 6) | (1000, 6) |